<a href="https://colab.research.google.com/github/SumeyyeSuslu/regression/blob/main/convertjson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu örnekte klasördeki bütün .plt dosyalarını birleştirerek tek bir DataFrame elde etme ve bunu json formatında kaydetme amaçlanmıştır. Daha önce plt dosyalarını binary olarak okuyarak içindeki verileri çıkartan pltreader.py dosyası construct library sini kullandığı için construct da kurulmuştur.

In [ ]:
!pip install construct

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for construct: filename=construct-2.10.68-py3-none-any.whl size=59226 sha256=f4d4bd10450780c77dbaffa587fa0617ad9c4f0dd1a091772d85035826b5e1c3
  Stored in directory: /root/.cache/pip/wheels/cf/58/27/8a7c57f4e85722ec338b53cef911ca3ebfe5e9f56c750532a2
Successfully built construct


In [ ]:
      #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZzZ1`1

Verilerin zipten çıkarılması

In [ ]:
!7z x DummyGeometryforPHDwork.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 30742711 bytes (30 MiB)

Extracting archive: DummyGeometryforPHDwork.7z
--
Path = DummyGeometryforPHDwork.7z
Type = 7z
Physical Size = 30742711
Headers Size = 1602
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      9% 26 - DummyGeometry/mach0.200_aoa10. . 00_aoa10.00_beta0.00-POINT.dat                                                                          17% 34 - DummyGeometry/mach0.200_aoa6.00 . ach0.200_aoa6.00_beta0-POINT.dat                                                             

DummyGeometry dizininin içindeki bütün .plt dosyaları taranarak adresleri bir list veri yapısına alınmıştır. 

In [ ]:
import os
file_paths = [] 
for root, dirs, files in os.walk("./DummyGeometry"):
    for file in files:
        if file.endswith(".plt"):
             file_paths.append(os.path.join(root, file))
print(file_paths)

Her bir .plt dosyasının dosya isminde hangi AoA, Mach ve beta değerinde analiz yapıldığının bilgisi bulunmaktadır. Bu sebeple dosya ismi okunarak bir Regex yardımıyla bu bilgilere ulaşılarak yeni feature olarak datasete eklenmektedir. Ayrıca her bir analizde her bir zone a ait veriler zone_id isimli bir feature kullanılarak işaretlenmiştir. Böylece ileride verisetini zone bazlı olarak filtrelemek veya zone bilgisini regresyon hesabında kullanmak mümkün olabilecektir. 

In [ ]:

import re
import numpy as np
import pltreader as plt

dataset = []
for item in file_paths:
  file_content={}
  pat = re.compile("mach(.*)_aoa(.*)_beta(.*)/")
  obj = pat.search(item)
  mach = float(obj.group(1))
  aoa = float(obj.group(2))
  beta = float(obj.group(3))
  with open( item, 'rb') as f:
    bytelist = f.read()
  header = plt.read_header(bytelist)
  data = plt.read_data(bytelist,header)
  for zone_id in range (len(data['Zones'])):
    zone_shape = data['Zones'][zone_id]['Pressure'].shape
    data['Zones'][zone_id]['mach']=mach*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['aoa']=aoa*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['beta']=beta*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['zone']=zone_id*np.ones(zone_shape, dtype = 'float32')

  file_content['data'] = data
  dataset.append(file_content)

In [ ]:
cols=['mach','aoa','beta','zone']+header['VarNames']
s={}
for col in cols:
  t=[]
  for fid in range(len(file_paths)):
    for zid in range(len(data['Zones'])):
      t.append(dataset[fid]['data']['Zones'][zid][col])
  s[col] = np.hstack(t)

Dict veri yapısında tutulan veriseti önce pandas dataframe, sonrasında JSON olarak kaydedilmiştir.

In [ ]:
import pandas as pd
pandasDF=pd.DataFrame.from_dict(s)
pandasDF.to_json('dummygeometry.json', orient='records', force_ascii=False, lines=True)